In [34]:
# univariate lstm example
import pandas as pd
import numpy as np
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed

In [35]:
events_dict = {
    'BLUE: first_blood': 0,
    'BLUE: dragon': 1,
    'BLUE: herald': 2,
    'BLUE: first_tower_top': 3,
    'BLUE: first_tower_mid': 4,
    'BLUE: first_tower_bot': 5,
    'BLUE: second_tower_top': 6,
    'BLUE: second_tower_mid': 7,
    'BLUE: second_tower_bot': 8,
    'BLUE: third_tower_top': 9,
    'BLUE: third_tower_mid': 10,
    'BLUE: third_tower_bot': 11,
    'BLUE: inhibitor_top': 12,
    'BLUE: inhibitor_mid': 13,
    'BLUE: inhibitor_bot': 14,
    'BLUE: baron': 15,
    'BLUE: elder_dragon': 16,
    'BLUE: nexus_tower': 17,
    'BLUE: nexus': 18,
    'RED: first_blood': 19,
    'RED: dragon': 20,
    'RED: herald': 21,
    'RED: first_tower_top': 22,
    'RED: first_tower_mid': 23,
    'RED: first_tower_bot': 24,
    'RED: second_tower_top': 25,
    'RED: second_tower_mid': 26,
    'RED: second_tower_bot': 27,
    'RED: third_tower_top': 28,
    'RED: third_tower_mid': 29,
    'RED: third_tower_bot': 30,
    'RED: inhibitor_top': 31,
    'RED: inhibitor_mid': 32,
    'RED: inhibitor_bot': 33,
    'RED: baron': 34,
    'RED: elder_dragon': 35,
    'RED: nexus_tower': 36,
    'RED: nexus': 37
}

n_in = 5
df = pd.read_csv('data/clean-one-line.csv')
df = df.drop('golId', axis=1)
data = df.values.tolist()

for game in data:
    for i, s in enumerate(game):
        if s in events_dict:
            game[i] = events_dict.get(s)

sequence = [b for b in data if not(isinstance(b, float))]
sequence = [[x for x in y if not np.isnan(x)] for y in sequence]

In [36]:
def splitSequence(games, n_steps_in, n_steps_out=1):
    X, Y = [], []
    
    for sequence in games:
        for i in range(len(sequence)):
            # find the end of this pattern
            end_ix = i + n_steps_in
            out_end_ix = end_ix + n_steps_out
            # check if we are beyond the sequence
            if out_end_ix > len(sequence):
                break
            # gather input and output parts of the pattern
            seq_x, seq_y = sequence[i:end_ix], sequence[end_ix:out_end_ix]
            X.append(seq_x)
            Y.append(seq_y)
	
    return array(X), array(Y)

In [37]:
X, y = splitSequence(sequence, n_in)
len(X)

62503

In [38]:
# choose a number of time steps
n_steps_in, n_steps_out = 5, 1
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
y = y.reshape((y.shape[0], y.shape[1], n_features))
# define model
model = Sequential()
model.add(LSTM(100, activation='softmax', input_shape=(n_steps_in, n_features)))
model.add(RepeatVector(n_steps_out))
model.add(LSTM(100, activation='softmax', return_sequences=True))
model.add(TimeDistributed(Dense(1)))
model.compile(optimizer='adam', loss='mse', metrics='accuracy')
# fit model
model.fit(X, y, epochs=10, verbose=1)

Epoch 1/10
1954/1954 [==============================] - 12s 5ms/step - loss: 376.0243 - accuracy: 0.0624
Epoch 2/10
1954/1954 [==============================] - 12s 6ms/step - loss: 270.5291 - accuracy: 0.0738
Epoch 3/10
1954/1954 [==============================] - 11s 6ms/step - loss: 200.7519 - accuracy: 0.0738
Epoch 4/10
1954/1954 [==============================] - 12s 6ms/step - loss: 156.7926 - accuracy: 0.0738
Epoch 5/10
1954/1954 [==============================] - 10s 5ms/step - loss: 133.8641 - accuracy: 0.0738
Epoch 6/10
1954/1954 [==============================] - 11s 5ms/step - loss: 126.0188 - accuracy: 0.0738
Epoch 7/10
1954/1954 [==============================] - 10s 5ms/step - loss: 121.8533 - accuracy: 0.0738
Epoch 8/10
1954/1954 [==============================] - 11s 5ms/step - loss: 108.8658 - accuracy: 0.0738
Epoch 9/10
1954/1954 [==============================] - 11s 6ms/step - loss: 98.5997 - accuracy: 0.0738
Epoch 10/10
1954/1954 [==============================] -

In [39]:
# demonstrate prediction
x_input = array([19, 1, 2, 1, 24])
x_input = x_input.reshape((1, n_steps_in, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)

[[[14.630391]]]
